In [1]:
from auxiliary import loadData_staticTargetAddrMatch, prepare_data_loaders
from algorithms import RssPosAlgo_NeuralNet_MLP4layer
from algorithms import RssPosAlgo_NeuralNet_supervisedTrainingLoop
from algorithms import RssPosAlgo_NearestNeighbour_Interpolation
from algorithms import RssPosAlgo_NearestNeighbour_GetKmeansDb
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

### prepare model, dataloaders and training parameters

In [2]:
datajsonpath = "../experiments/exp002_sna_kadirburak/data-tshark/data.json"
inp_rss_vals, gt_locations = loadData_staticTargetAddrMatch(datajsonpath, second_hold = 5, shuffle=False, 
                                                            target_addresses=["4c:77:6d:92:b3:60",
"4c:77:6d:5f:ea:e0",
"4c:77:6d:05:af:20"])

epochs           = 601
batch_size       = 8
train_test_split = 0.8

train_loader, test_loader, xtr, ytr, xts, yts = prepare_data_loaders(inp_rss_vals, gt_locations, 
                                                                     batch_size = 32, 
                                                                     train_test_split = train_test_split)

print("Subset sizes | train:", xtr.shape[0], ", test:",xts.shape[0])
MLP = RssPosAlgo_NeuralNet_MLP4layer()
MLP.train()

MLP_criterion = nn.MSELoss(reduction='mean')
MLP_optimizer = optim.Adam(MLP.parameters(), lr=3e-4)

Subset sizes | train: 278 , test: 70


### train model

In [3]:
MLP = RssPosAlgo_NeuralNet_supervisedTrainingLoop(train_loader = train_loader, 
                                                         test_loader  = test_loader,
                                                         model        = MLP, 
                                                         criterion    = MLP_criterion, 
                                                         optimizer    = MLP_optimizer, 
                                                         epochs       = epochs,
                                                         testfreq     = 20) # testfreq is in epochs

Epoch [1/601] test loss: 46.650, training loss: -1.000
Epoch [21/601] test loss: 1.053, training loss: 11.465
Epoch [41/601] test loss: 0.970, training loss: 10.931
Epoch [61/601] test loss: 0.922, training loss: 10.605
Epoch [81/601] test loss: 0.889, training loss: 10.335
Epoch [101/601] test loss: 0.866, training loss: 10.113
Epoch [121/601] test loss: 0.853, training loss: 9.935
Epoch [141/601] test loss: 0.847, training loss: 9.795
Epoch [161/601] test loss: 0.848, training loss: 9.684
Epoch [181/601] test loss: 0.851, training loss: 9.599
Epoch [201/601] test loss: 0.858, training loss: 9.528
Epoch [221/601] test loss: 0.865, training loss: 9.467
Epoch [241/601] test loss: 0.872, training loss: 9.412
Epoch [261/601] test loss: 0.880, training loss: 9.363
Epoch [281/601] test loss: 0.892, training loss: 9.315
Epoch [301/601] test loss: 0.906, training loss: 9.271
Epoch [321/601] test loss: 0.916, training loss: 9.237
Epoch [341/601] test loss: 0.926, training loss: 9.211
Epoch [36

### save the model

In [4]:
torch.save(MLP.state_dict(), 'savedmodels/dev004_exp002_MLP4layer.pth')

### evaluate model

In [5]:
MLP = RssPosAlgo_NeuralNet_MLP4layer()
MLP.load_state_dict(torch.load('savedmodels/dev004_exp002_MLP4layer.pth'));
MLP.eval();

/tmp/ipykernel_7318/2698718012.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  MLP.load_state_dict(torch.load('savedmodels/dev004_exp002_MLP4layer.pth'));


In [6]:
db_kmeans = RssPosAlgo_NearestNeighbour_GetKmeansDb(xtr, ytr, num_clusters=3)
meanerror_nene_interp = 0;
meanerror_mlp         = 0;
for test_idx, x_test_sample in enumerate(xts): 
    loc_pred_mlp           = MLP(x_test_sample)
    loc_pred_nene_interp   = RssPosAlgo_NearestNeighbour_Interpolation(x_test_sample, db_kmeans)
    meanerror_mlp         += (yts[test_idx].numpy() - loc_pred_mlp.detach().numpy())**2
    meanerror_nene_interp += (yts[test_idx].numpy() - loc_pred_nene_interp)**2
print("MLP        :", np.linalg.norm(meanerror_mlp/(test_idx+1)))
print("NeNe+Interp:", np.linalg.norm(meanerror_nene_interp/(test_idx+1)))

MLP        : 0.58780384
NeNe+Interp: 0.5789257631155487


### test a few samples

from test set

In [7]:
sampleid = 10
loc_pred_mlp           = MLP(xts[sampleid])
loc_pred_nene_interp   = RssPosAlgo_NearestNeighbour_Interpolation(xts[sampleid], db_kmeans)
meanerror_mlp         += (yts[sampleid].numpy() - loc_pred_mlp.detach().numpy())**2
meanerror_nene_interp += (yts[sampleid].numpy() - loc_pred_nene_interp)**2
print("Actual position  :", yts[sampleid].numpy())
print("MLP prediction   :", loc_pred_mlp.detach().numpy())
print("NeNe+Interp pred :", loc_pred_nene_interp)

Actual position  : [0.5928982 3.92059  ]
MLP prediction   : [1.474773 2.341582]
NeNe+Interp pred : (1.735097671019228, 2.370105143219773)


---

from train set

In [8]:
sampleid = 20
loc_pred_mlp           = MLP(xtr[sampleid])
loc_pred_nene_interp   = RssPosAlgo_NearestNeighbour_Interpolation(xtr[sampleid], db_kmeans)
meanerror_mlp         += (ytr[sampleid].numpy() - loc_pred_mlp.detach().numpy())**2
meanerror_nene_interp += (ytr[sampleid].numpy() - loc_pred_nene_interp)**2
print("Actual position  :", ytr[sampleid].numpy())
print("MLP prediction   :", loc_pred_mlp.detach().numpy())
print("NeNe+Interp pred :", loc_pred_nene_interp)

Actual position  : [2.79366   3.6832027]
MLP prediction   : [2.1297522 3.052216 ]
NeNe+Interp pred : (2.331297082015229, 3.0681399293100764)
